In [1]:
# !gdown https://drive.google.com/file/d/1Hv4RAltBumSfOkRacoX8qrfDYfd_NDss/view?usp=drive_link --fuzzy

Traceback (most recent call last):
  File "/raid/home/niranjan20090/anaconda3/envs/tiger/bin/gdown", line 8, in <module>
    sys.exit(main())
  File "/raid/home/niranjan20090/anaconda3/envs/tiger/lib/python3.9/site-packages/gdown/cli.py", line 156, in main
    filename = download(
  File "/raid/home/niranjan20090/anaconda3/envs/tiger/lib/python3.9/site-packages/gdown/download.py", line 259, in download
    filename_from_url = m.groups()[0]
AttributeError: 'NoneType' object has no attribute 'groups'


In [2]:
# !unzip Dataset_AML_Assignment1_Part1.zip

Archive:  Dataset_AML_Assignment1_Part1.zip
  inflating: df_syn_train_0_0_.csv   
  inflating: df_synA_test_hard_shuffled_sample.csv  


  inflating: df_synA_train_shuffled.csv  


In [4]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn.functional as F
from IPython import display
from tqdm.notebook import tqdm
import random
import math, time, os
from matplotlib import pyplot as plt
import pickle as pkl


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
prefix = './data_dump_selective_classification/'


In [6]:
# NAL - ninju
# Ensemble - mudit
# Autoencoder - ninju
# SubTab
# Meta Learning (clean+dirty)

In [2]:
class SinusodialDataset(Dataset):
    def __init__(self, df, mode='era'):
        """ creating label columns of eras and targets """
        self.X = df.iloc[:, :24]
        if mode == 'era':
          self.y = df['era_label']
        else:
          self.y = df[f'{mode}']

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        X = torch.tensor(self.X.iloc[idx].values, dtype=torch.float32)
        y = torch.tensor(int(self.y.iloc[idx]), dtype=torch.long)
        return X, y

In [3]:
def make_data_splits(df, mode, batch_size=32, train_perc=0.7, val_test_perc=0.5):

    def encode(v, class_values):
        return class_values.index(v)

    #adding new era_label column indexed 0, 1,...
    class_values = df['era'].unique().tolist()
    df['era_label'] = df['era'].apply(lambda x: encode(x, class_values))
    df.reset_index(drop=True, inplace=True)

    train_samples = int(len(df)*train_perc)
    val_test_samples = len(df)-train_samples

    data = SinusodialDataset(df, mode=mode)
    data_train, data_test = random_split(data, [train_samples, val_test_samples])

    val_samples = int(len(data_test)*0.5)
    test_samples = len(data_test)-val_samples
    data_val, data_test = random_split(data_test, [val_samples, test_samples])

    print("Train-val-test lengths: ", len(data_train), len(data_val), len(data_test))

    loader_train = DataLoader(data_train, batch_size=batch_size, shuffle=True)
    loader_val = DataLoader(data_val, batch_size=batch_size, shuffle=False)
    loader_test = DataLoader(data_test, batch_size=batch_size, shuffle=False)

    return loader_train, loader_val, loader_test, data

#DDT

In [19]:
class DifferentiableDecisionTree(nn.Module):
    def __init__(self, input_dim, output_dim, max_depth=3):
        super(DifferentiableDecisionTree, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.max_depth = max_depth

        # Parameters for decision rules
        self.feature_threshold = nn.Parameter(torch.randn(max_depth, input_dim))
        self.left_weights = nn.Parameter(torch.randn(max_depth, output_dim, input_dim))
        self.right_weights = nn.Parameter(torch.randn(max_depth, output_dim, input_dim))

        self.optimizer = optim.SGD(self.parameters(), lr=0.01)


    def forward(self, x):
        # Initialize leaf node values to zero
        leaf_values = torch.zeros(x.shape[0], self.output_dim).to(device)

        for d in range(self.max_depth):
            # Compute decision rule
            decision = x[:, None, :] < self.feature_threshold[d]
            # Apply decision rule to update leaf node values
            left_values = torch.mul(decision.float(), self.left_weights[d]).sum(dim=-1)
            right_values = torch.mul((1 - decision.float()), self.right_weights[d]).sum(dim=-1)
            leaf_values = leaf_values + left_values + right_values

        return leaf_values



In [5]:
def accuracy(Net, X_test, y_test, verbose=True):
    Net.eval()
    m = X_test.shape[0]
    y_pred = Net(X_test)
    predicted = torch.max(y_pred, 1)[1]
    correct = (predicted == y_test).float().sum().item()
    if verbose:
        print(correct,m)
    accuracy = correct/m
    Net.train()
    return accuracy, correct

In [6]:
def Test(net, loader_test, mode, noise_level, \
         device='cpu', Loss=nn.CrossEntropyLoss(reduction='sum')):
    net.eval()
    total_samples = 0
    correct_samples = 0
    loss = 0.0
    for (X, y) in loader_test:
        X=X.to(device)
        y=y.to(device)
        total_samples += y.shape[0]
        _, i_cor_sam = accuracy(net,X,y,verbose=False)
        correct_samples += i_cor_sam
        loss += Loss(net(X), y).cpu().detach().item()
    acc = correct_samples / total_samples
    loss /= total_samples
    return loss, acc

In [7]:
def Train(Net, data, mode, noise_level, epochs=20, lr=5e-2, Loss=nn.CrossEntropyLoss(reduction='sum'), verbose=False, device='cpu',
          val_ds=None, plot_accs=False, plot_losses=False):
    model_save_time = time.time()
    losses = []
    accs = []
    val_losses=[]
    val_accL=[]
    Net.to(device)
    for e in range(epochs):
        Net.train()
        step=0
        tot_loss=0.0
        start_time = time.time()
        correct_samples = 0
        total_samples = 0
        for (X,y) in data:
            X=X.to(device)
            y=y.to(device)
            total_samples += y.shape[0]
            y_pred = Net(X)
            loss = Loss(y_pred,y)
            Net.optimizer.zero_grad()
            loss.backward()
            Net.optimizer.step()
            step+=1
            tot_loss+=loss
            if verbose:
                _, i_cor_sam = accuracy(Net,X,y,verbose=False)
                correct_samples += i_cor_sam
        end_time = time.time()
        t = end_time-start_time
        l = tot_loss.item()/total_samples
        losses += [l]
        if verbose:
            a = correct_samples/total_samples
            accs += [a]
            print('Epoch %2d Loss: %2.5e Accuracy: %2.5f Epoch Time: %2.5f' %(e,l,a,t))

        val_loss, val_acc = Test(Net, val_ds, mode, noise_level, device)
        val_losses.append(val_loss)
        val_accL.append(val_acc)

        torch.save(Net.state_dict(), f'{prefix}/net_{noise_level}_{mode}_{str(model_save_time)}.pth')

    return Net, losses, accs, val_losses, val_accL

In [8]:
def plot_loss_acc(losses, accs, val_losses, val_accs, mode, noise_level):

    plt.plot(np.array(accs),color='red', label='Train accuracy')
    plt.plot(np.array(val_accs),color='blue', label='Val accuracy')
    plt.legend()
    plt.savefig(f'{prefix}/acc_{mode}_{noise_level}.png')
    plt.clf()

    plt.plot(np.array(losses),color='red', label='Train loss')
    plt.plot(np.array(val_losses),color='blue', label='Val loss')
    plt.legend()
    plt.savefig(f'{prefix}/loss_{mode}_{noise_level}.png')
    plt.clf()
    return

In [21]:
df_paths = ['../Datasets/df_syn_train_0_0_.csv',
            '../Datasets/df_synA_train_shuffled.csv',
            '../Datasets/df_synA_test_hard_shuffled_sample.csv']

noise_levels = ['none', 'low', 'high']
batch_sizes = [32, 128, 128]

losses_arr = []
accs_arr = []

for i in range(0, 3):
    df = pd.read_csv(df_paths[i])

    modes = ['era', 'target_5_val', 'target_10_val']

    loss_per_mode = []
    acc_per_mode = []

    for mode in modes:
        print("Noise Level:", noise_levels[i], "Mode:", mode)
        loader_train, loader_val, loader_test, data = make_data_splits(df, mode=mode, \
                                                                       batch_size=batch_sizes[i])
        # net = MLP(dims=[data.X.shape[1], 32, 64, 32, len(data.y.unique())]).to(device)
        net = DifferentiableDecisionTree(input_dim=data.X.shape[1], output_dim=len(data.y.unique())).to(device)

        net, losses, accs, val_losses, val_accL = Train(net, loader_train, mode, noise_levels[i], \
                                      epochs=50, verbose=True, device=device, val_ds=loader_val, \
                                      plot_accs=True, plot_losses=True)
        plot_loss_acc(losses, accs, val_losses, val_accL, mode, noise_levels[i])
        #Testing code
        test_loss, test_acc = Test(net, loader_test, mode, noise_levels[i], device=device)
        print(test_loss, test_acc)
        loss_per_mode.append(test_loss)
        acc_per_mode.append(test_acc)

    losses_arr.append(loss_per_mode)
    accs_arr.append(acc_per_mode)

Noise Level: none Mode: era
Train-val-test lengths:  5460 1170 1170
Epoch  0 Loss: 2.83893e+00 Accuracy: 0.36777 Epoch Time: 0.78098
Epoch  1 Loss: 1.58776e+00 Accuracy: 0.58223 Epoch Time: 0.76602
Epoch  2 Loss: 1.41947e+00 Accuracy: 0.62564 Epoch Time: 0.76216
Epoch  3 Loss: 1.34812e+00 Accuracy: 0.64084 Epoch Time: 0.76531
Epoch  4 Loss: 1.24406e+00 Accuracy: 0.65934 Epoch Time: 0.75868
Epoch  5 Loss: 1.25199e+00 Accuracy: 0.66886 Epoch Time: 0.77805
Epoch  6 Loss: 1.17619e+00 Accuracy: 0.67656 Epoch Time: 0.76348
Epoch  7 Loss: 1.16760e+00 Accuracy: 0.67473 Epoch Time: 0.79526
Epoch  8 Loss: 1.15853e+00 Accuracy: 0.66923 Epoch Time: 0.76251
Epoch  9 Loss: 1.14546e+00 Accuracy: 0.67674 Epoch Time: 0.75744
Epoch 10 Loss: 1.12980e+00 Accuracy: 0.68168 Epoch Time: 0.78972
Epoch 11 Loss: 1.11499e+00 Accuracy: 0.68938 Epoch Time: 0.80008
Epoch 12 Loss: 1.09204e+00 Accuracy: 0.69304 Epoch Time: 0.79785
Epoch 13 Loss: 1.12878e+00 Accuracy: 0.68883 Epoch Time: 0.78597
Epoch 14 Loss: 1.12425

<Figure size 640x480 with 0 Axes>

In [22]:
with open(f'{prefix}/losses_dump.pkl', 'wb') as f:
    pkl.dump(losses_arr, f)

with open(f'{prefix}/accs_dump.pkl', 'wb') as f:
    pkl.dump(accs_arr, f)